<a href="https://colab.research.google.com/github/gitmystuff/DTSC5082/blob/main/Interview_Prep_3/interview_prep_simulation_scenarios_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AB Testing Assignment: Optimizing E-Commerce Conversion Rates

## Assignment Overview

**Estimated Time**: 60 minutes

**Learning Objectives**:
- Design and conduct a complete AB test from start to finish
- Formulate null and alternative hypotheses
- Calculate and interpret test statistics and p-values
- Make data-driven decisions using statistical evidence
- Understand the difference between statistical and practical significance
- Communicate results effectively

---

## Business Scenario

You are a data scientist at **ShopSmart**, a growing e-commerce company. The product team has redesigned the checkout button on the website, changing it from a standard blue button labeled "Proceed to Checkout" to a more prominent green button labeled "Complete My Purchase".

### The Question
**Does the new button design increase the checkout conversion rate?**

### Current Performance
- The current checkout button has a historical conversion rate of approximately **12%**
- Management wants to see at least a **2 percentage point improvement** (to 14%) to justify the change
- They want to be 95% confident in the results (α = 0.05)

### The Test Setup
- **Control Group (A)**: Users see the old blue button
- **Treatment Group (B)**: Users see the new green button
- You'll randomly assign visitors to each group and track whether they complete checkout

---

## Part 1: Setup and Data Generation (10 minutes)

### Task 1.1: Import Libraries and Set Seed

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, binom
import time
from datetime import datetime

# Set style for better-looking plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Generate time-based seed for reproducibility
# This creates a unique seed each time you run it, but you can reproduce results using the printed seed
seed = int(time.time() * 1000) % 100000  # Use milliseconds, mod to keep it reasonable
np.random.seed(seed)

print("="*70)
print("AB TEST SIMULATION SETUP")
print("="*70)
print(f"Random Seed: {seed}")
print(f"Date/Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n⚠️ IMPORTANT: Save this seed number to reproduce your exact results!")
print("="*70)

### Task 1.2: Formulate Your Hypotheses

Before collecting data, we need to clearly state what we're testing.

**✏️ YOUR TASK**: Complete the hypotheses below by filling in the blanks.

Let:
- p_A = conversion rate of Control group (old button)
- p_B = conversion rate of Treatment group (new button)

**Instructions**: Write your hypotheses in the markdown cell below.

**YOUR ANSWER HERE:**

**Null Hypothesis (H₀)**:
_[Write what you're testing against - hint: is there a difference?]_

**Alternative Hypothesis (Hₐ)**:
_[Write what you hope to find - hint: is the new button better?]_

**Significance Level (α)**:
_[What is our threshold for rejecting H₀?]_

**Type of Test**:
_[One-tailed or two-tailed? Why?]_

### Task 1.3: Generate Simulated AB Test Data

We'll simulate data for our AB test. In a real scenario, this would be actual user behavior data.

**✏️ YOUR TASK**: Run the following code to generate your dataset. Read the comments to understand what's happening.

In [ ]:
# Define the true underlying conversion rates (unknown to us as analysts)
# In reality, we're testing to discover these!
true_control_rate = 0.12  # 12% baseline
true_treatment_rate = 0.145  # 14.5% with new button (2.5 percentage point lift)

# Sample sizes for each group
n_control = 2000
n_treatment = 2000

# Generate conversion data (1 = converted, 0 = did not convert)
# We use binomial distribution: each user is a Bernoulli trial
control_conversions = np.random.binomial(1, true_control_rate, n_control)
treatment_conversions = np.random.binomial(1, true_treatment_rate, n_treatment)

# Create a DataFrame
df_control = pd.DataFrame({
    'user_id': range(1, n_control + 1),
    'group': 'Control',
    'converted': control_conversions
})

df_treatment = pd.DataFrame({
    'user_id': range(n_control + 1, n_control + n_treatment + 1),
    'group': 'Treatment',
    'converted': treatment_conversions
})

# Combine into one dataset
df = pd.concat([df_control, df_treatment], ignore_index=True)

# Shuffle the dataset (as users would arrive randomly)
df = df.sample(frac=1, random_state=seed).reset_index(drop=True)

print("\n📊 Data Generated Successfully!")
print(f"Total users: {len(df):,}")
print(f"Control group: {len(df[df['group']=='Control']):,}")
print(f"Treatment group: {len(df[df['group']=='Treatment']):,}")
print("\nFirst few rows:")
print(df.head(10))

# Note: In a real AB test, you wouldn't know the true rates!
# We're simulating data, so we set them for the simulation.

---

## Part 2: Exploratory Data Analysis (10 minutes)

### Task 2.1: Calculate Sample Statistics

**✏️ YOUR TASK**: Calculate the conversion rates and counts for each group.

In [ ]:
# Calculate statistics for each group
# YOUR CODE HERE: Fill in the blanks

# Control group
n_A = len(df[df['group'] == 'Control'])  # Sample size
conversions_A = df[df['group'] == 'Control']['converted'].sum()  # Number of conversions
p_A = ___  # Conversion rate (hint: conversions / sample size)

# Treatment group
n_B = len(df[df['group'] == 'Treatment'])
conversions_B = df[df['group'] == 'Treatment']['converted'].sum()
p_B = ___  # Conversion rate

# Observed difference
observed_diff = p_B - p_A

print("="*70)
print("SAMPLE STATISTICS")
print("="*70)
print(f"\nControl Group (A - Old Button):")
print(f"  Sample size: {n_A:,}")
print(f"  Conversions: {conversions_A:,}")
print(f"  Conversion rate: {p_A:.4f} ({p_A*100:.2f}%)")
print(f"\nTreatment Group (B - New Button):")
print(f"  Sample size: {n_B:,}")
print(f"  Conversions: {conversions_B:,}")
print(f"  Conversion rate: {p_B:.4f} ({p_B*100:.2f}%)")
print(f"\nObserved Difference (B - A):")
print(f"  Absolute: {observed_diff:.4f} ({observed_diff*100:.2f} percentage points)")
print(f"  Relative: {(observed_diff/p_A)*100:.2f}% lift")
print("="*70)

### Task 2.2: Visualize the Results

**✏️ YOUR TASK**: Create a bar chart comparing conversion rates.

In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Conversion rates
groups = ['Control\n(Old Button)', 'Treatment\n(New Button)']
rates = [p_A, p_B]
colors = ['#3498db', '#2ecc71']

bars = ax1.bar(groups, rates, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax1.set_ylabel('Conversion Rate', fontsize=12)
ax1.set_title('Conversion Rates by Group', fontsize=14, fontweight='bold')
ax1.set_ylim(0, max(rates) * 1.3)

# Add value labels on bars
for bar, rate in zip(bars, rates):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{rate*100:.2f}%',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

# Add horizontal line for historical baseline
ax1.axhline(y=0.12, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Historical Baseline (12%)')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Conversion counts
conv_data = pd.DataFrame({
    'Group': ['Control']*2 + ['Treatment']*2,
    'Status': ['Converted', 'Did Not Convert']*2,
    'Count': [conversions_A, n_A - conversions_A, conversions_B, n_B - conversions_B]
})

# Pivot for stacked bar chart
pivot_data = conv_data.pivot(index='Group', columns='Status', values='Count')
pivot_data.plot(kind='bar', stacked=True, ax=ax2, color=['#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
ax2.set_ylabel('Number of Users', fontsize=12)
ax2.set_title('Conversion Breakdown by Group', fontsize=14, fontweight='bold')
ax2.set_xlabel('')
ax2.set_xticklabels(['Control', 'Treatment'], rotation=0)
ax2.legend(title='Status')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

**🤔 REFLECTION QUESTION**: Based on the visualizations, does it appear that the new button performs better? Write your initial thoughts below before conducting the formal statistical test.

**YOUR ANSWER HERE:**

_[Your initial thoughts about whether the new button seems to be working]_

---

## Part 3: Hypothesis Testing (20 minutes)

### Task 3.1: Calculate the Pooled Proportion and Standard Error

For a two-proportion z-test, we calculate a pooled proportion under the null hypothesis (that both groups have the same conversion rate).

**Formulas**:
- Pooled proportion: p_pool = (conversions_A + conversions_B) / (n_A + n_B)
- Standard Error: SE = sqrt(p_pool × (1 - p_pool) × (1/n_A + 1/n_B))

**✏️ YOUR TASK**: Calculate these values.

In [ ]:
# YOUR CODE HERE

# Calculate pooled proportion
p_pool = (conversions_A + conversions_B) / (n_A + n_B)

# Calculate standard error
SE = np.sqrt(p_pool * (1 - p_pool) * (1/n_A + 1/n_B))

print("="*70)
print("POOLED STATISTICS")
print("="*70)
print(f"Pooled proportion: {p_pool:.4f} ({p_pool*100:.2f}%)")
print(f"Standard Error: {SE:.4f}")
print("="*70)

### Task 3.2: Calculate the Test Statistic

The test statistic (z-score) measures how many standard errors our observed difference is from zero (the null hypothesis).

**Formula**: z = (p_B - p_A) / SE

**✏️ YOUR TASK**: Calculate the z-statistic.

In [ ]:
# YOUR CODE HERE
z_statistic = ___  # Fill in the formula

print("="*70)
print("TEST STATISTIC")
print("="*70)
print(f"Z-statistic: {z_statistic:.4f}")
print(f"\nInterpretation: The observed difference is {abs(z_statistic):.2f} standard errors")
print(f"{'above' if z_statistic > 0 else 'below'} what we'd expect if there were no true difference.")
print("="*70)

### Task 3.3: Calculate P-Value and Critical Value

**✏️ YOUR TASK**: Calculate the p-value and critical value for α = 0.05.

In [ ]:
# Set significance level
alpha = 0.05

# Calculate critical value for one-tailed test
# (We're testing if treatment > control)
z_critical = norm.ppf(1 - alpha)  # Right tail

# Calculate p-value (one-tailed)
p_value = 1 - norm.cdf(z_statistic)  # Right tail

print("="*70)
print("HYPOTHESIS TEST RESULTS")
print("="*70)
print(f"Significance level (α): {alpha}")
print(f"Critical value (z): {z_critical:.4f}")
print(f"Test statistic (z): {z_statistic:.4f}")
print(f"P-value: {p_value:.6f}")
print("\n" + "-"*70)
print("DECISION RULE")
print("-"*70)
print(f"If p-value < α ({alpha}), reject H₀")
print(f"If z-statistic > z-critical ({z_critical:.4f}), reject H₀")
print("="*70)

### Task 3.4: Visualize the Test

**✏️ YOUR TASK**: Create a visualization showing the null distribution, critical region, and test statistic.

In [ ]:
# Create visualization of hypothesis test
fig, ax = plt.subplots(figsize=(12, 7))

# Create x-axis values
x = np.linspace(-4, 4, 1000)
y = norm.pdf(x, 0, 1)  # Standard normal distribution

# Plot the null distribution
ax.plot(x, y, 'b-', linewidth=2, label='Null Distribution (H₀: p_B = p_A)')

# Fill rejection region (right tail for one-tailed test)
x_reject = x[x >= z_critical]
y_reject = norm.pdf(x_reject, 0, 1)
ax.fill_between(x_reject, y_reject, alpha=0.3, color='red',
                label=f'Rejection Region (α = {alpha})')

# Mark critical value
ax.axvline(z_critical, color='red', linestyle='--', linewidth=2,
           label=f'Critical Value = {z_critical:.3f}')

# Mark observed test statistic
ax.axvline(z_statistic, color='green', linestyle='-', linewidth=3,
           label=f'Test Statistic = {z_statistic:.3f}')

# Fill p-value area
x_pval = x[x >= z_statistic]
y_pval = norm.pdf(x_pval, 0, 1)
ax.fill_between(x_pval, y_pval, alpha=0.5, color='yellow',
                label=f'P-value = {p_value:.4f}')

# Labels and formatting
ax.set_xlabel('Z-score', fontsize=12)
ax.set_ylabel('Probability Density', fontsize=12)
ax.set_title('Hypothesis Test Visualization (One-Tailed)', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)
ax.grid(alpha=0.3)
ax.set_xlim(-4, 4)

# Add decision text
decision_text = "REJECT H₀" if p_value < alpha else "FAIL TO REJECT H₀"
decision_color = 'green' if p_value < alpha else 'red'
ax.text(0.5, 0.95, f'Decision: {decision_text}',
        transform=ax.transAxes, fontsize=14, fontweight='bold',
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor=decision_color, alpha=0.3))

plt.tight_layout()
plt.show()

### Task 3.5: Make Your Decision

**✏️ YOUR TASK**: Complete the decision analysis below.

In [ ]:
# Make decision
print("\n" + "="*70)
print("STATISTICAL DECISION")
print("="*70)

if p_value < alpha:
    print("✓ REJECT the null hypothesis")
    print(f"\nReason: p-value ({p_value:.6f}) < α ({alpha})")
    print(f"Also: z-statistic ({z_statistic:.4f}) > critical value ({z_critical:.4f})")
    print("\nConclusion:")
    print(f"There IS statistically significant evidence (at the {alpha} level)")
    print(f"that the new button increases conversion rates.")
    print(f"\nThe new button showed a conversion rate of {p_B*100:.2f}%")
    print(f"compared to {p_A*100:.2f}% for the old button.")
    print(f"This represents a {observed_diff*100:.2f} percentage point increase.")
else:
    print("✗ FAIL TO REJECT the null hypothesis")
    print(f"\nReason: p-value ({p_value:.6f}) ≥ α ({alpha})")
    print(f"Also: z-statistic ({z_statistic:.4f}) ≤ critical value ({z_critical:.4f})")
    print("\nConclusion:")
    print(f"There is NOT statistically significant evidence (at the {alpha} level)")
    print(f"that the new button increases conversion rates.")
    print(f"\nWhile the new button showed {p_B*100:.2f}% vs {p_A*100:.2f}%,")
    print(f"this difference could reasonably be due to random chance.")

print("="*70)

---

## Part 4: Confidence Intervals (10 minutes)

### Task 4.1: Calculate 95% Confidence Interval for the Difference

A confidence interval gives us a range of plausible values for the true difference in conversion rates.

**Formula**: (p_B - p_A) ± z* × SE

**✏️ YOUR TASK**: Calculate the 95% confidence interval.

In [ ]:
# YOUR CODE HERE

# For 95% CI, we need z* for two-tailed (since CI is always two-sided)
confidence_level = 0.95
z_star = norm.ppf(1 - (1 - confidence_level)/2)  # Two-tailed

# Margin of error
margin_of_error = z_star * SE

# Confidence interval
ci_lower = observed_diff - margin_of_error
ci_upper = observed_diff + margin_of_error

print("="*70)
print("95% CONFIDENCE INTERVAL FOR DIFFERENCE IN CONVERSION RATES")
print("="*70)
print(f"Observed difference: {observed_diff:.4f} ({observed_diff*100:.2f}%)")
print(f"Margin of error: ± {margin_of_error:.4f}")
print(f"\n95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"In percentage points: [{ci_lower*100:.2f}%, {ci_upper*100:.2f}%]")
print("\nInterpretation:")
print(f"We are 95% confident that the true difference in conversion rates")
print(f"is between {ci_lower*100:.2f} and {ci_upper*100:.2f} percentage points.")

if ci_lower > 0:
    print(f"\n✓ The entire interval is above 0, supporting that B > A.")
elif ci_upper < 0:
    print(f"\n✗ The entire interval is below 0, suggesting A > B.")
else:
    print(f"\n⚠️ The interval contains 0, so we can't rule out no difference.")

print("="*70)

### Task 4.2: Visualize the Confidence Interval

In [ ]:
# Visualize confidence interval
fig, ax = plt.subplots(figsize=(12, 6))

# Plot point estimate
ax.scatter([observed_diff*100], [1], s=200, color='blue', zorder=5, label='Observed Difference')

# Plot confidence interval
ax.plot([ci_lower*100, ci_upper*100], [1, 1], 'b-', linewidth=4, label='95% CI')
ax.plot([ci_lower*100, ci_lower*100], [0.95, 1.05], 'b-', linewidth=4)
ax.plot([ci_upper*100, ci_upper*100], [0.95, 1.05], 'b-', linewidth=4)

# Add vertical line at 0 (null hypothesis)
ax.axvline(0, color='red', linestyle='--', linewidth=2, label='H₀: No Difference', alpha=0.7)

# Add vertical line at 2% (practical significance threshold)
ax.axvline(2, color='green', linestyle='--', linewidth=2, label='Target: 2% Improvement', alpha=0.7)

# Formatting
ax.set_xlabel('Difference in Conversion Rates (percentage points)', fontsize=12)
ax.set_ylabel('')
ax.set_title('95% Confidence Interval for Conversion Rate Difference', fontsize=14, fontweight='bold')
ax.set_ylim(0.5, 1.5)
ax.set_yticks([])
ax.legend(fontsize=11)
ax.grid(axis='x', alpha=0.3)

# Add annotation
ax.annotate(f'{observed_diff*100:.2f}%\n[{ci_lower*100:.2f}%, {ci_upper*100:.2f}%]',
            xy=(observed_diff*100, 1), xytext=(observed_diff*100, 1.3),
            ha='center', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', lw=2))

plt.tight_layout()
plt.show()

---

## Part 5: Business Interpretation and Practical Significance (10 minutes)

### Task 5.1: Statistical vs Practical Significance

Just because a result is *statistically significant* doesn't mean it's *practically significant* for the business.

**✏️ YOUR TASK**: Answer the following questions in the markdown cell below.

**YOUR ANSWERS HERE:**

1. **Is the result statistically significant?** (Yes/No and why)
   
   _[Your answer]_

2. **Does the confidence interval include the 2 percentage point target improvement?**
   
   _[Your answer]_

3. **Based on your analysis, should ShopSmart implement the new button? Why or why not?**
   
   _[Your answer - consider both statistical and practical significance]_

4. **What are the potential risks of your recommendation?**
   
   _[Your answer - think about Type I and Type II errors]_

### Task 5.2: Calculate the Practical Impact

Let's quantify what this improvement means in business terms.

**✏️ YOUR TASK**: Complete the business impact analysis.

In [ ]:
# Business impact calculation
# Assumptions (you can modify these)
monthly_visitors = 100000  # Number of visitors who reach checkout page per month
average_order_value = 75  # Average purchase amount in dollars

# Calculate impact
current_monthly_conversions = monthly_visitors * p_A
new_monthly_conversions = monthly_visitors * p_B
additional_conversions = new_monthly_conversions - current_monthly_conversions

current_monthly_revenue = current_monthly_conversions * average_order_value
new_monthly_revenue = new_monthly_conversions * average_order_value
additional_revenue = new_monthly_revenue - current_monthly_revenue

# Annual projections
annual_additional_revenue = additional_revenue * 12

print("="*70)
print("BUSINESS IMPACT ANALYSIS")
print("="*70)
print(f"\nAssumptions:")
print(f"  Monthly checkout page visitors: {monthly_visitors:,}")
print(f"  Average order value: ${average_order_value}")
print(f"\nCurrent Performance (Control):")
print(f"  Conversion rate: {p_A*100:.2f}%")
print(f"  Monthly conversions: {current_monthly_conversions:,.0f}")
print(f"  Monthly revenue: ${current_monthly_revenue:,.2f}")
print(f"\nProjected Performance (Treatment):")
print(f"  Conversion rate: {p_B*100:.2f}%")
print(f"  Monthly conversions: {new_monthly_conversions:,.0f}")
print(f"  Monthly revenue: ${new_monthly_revenue:,.2f}")
print(f"\nImpact of New Button:")
print(f"  Additional conversions per month: {additional_conversions:,.0f}")
print(f"  Additional revenue per month: ${additional_revenue:,.2f}")
print(f"  Additional revenue per year: ${annual_additional_revenue:,.2f}")
print(f"  Percentage increase in revenue: {(additional_revenue/current_monthly_revenue)*100:.2f}%")
print("="*70)

---

## Part 6: Power Analysis and Sample Size (Bonus - if time permits)

### Task 6.1: Understanding Statistical Power

**Statistical Power** is the probability of detecting a true effect when it exists (1 - β, where β is Type II error rate).

Let's calculate the power of our test.

In [ ]:
# Calculate statistical power
# Power = P(reject H₀ | H₁ is true)

# Under the alternative hypothesis, the distribution is centered at the true difference
# with the same standard error
true_diff = p_B - p_A  # This is what we observed (treating it as the true effect)

# Calculate z-score of the critical value under the alternative distribution
z_under_alt = (z_critical * SE - true_diff) / SE

# Power is the probability of being in the rejection region under H₁
power = 1 - norm.cdf(z_under_alt)

print("="*70)
print("STATISTICAL POWER ANALYSIS")
print("="*70)
print(f"Sample size per group: {n_A}")
print(f"Observed effect size: {observed_diff*100:.2f} percentage points")
print(f"Significance level (α): {alpha}")
print(f"\nStatistical Power: {power:.4f} ({power*100:.2f}%)")
print(f"Type II Error Rate (β): {1-power:.4f} ({(1-power)*100:.2f}%)")
print("\nInterpretation:")
print(f"If the true effect is {observed_diff*100:.2f} percentage points,")
print(f"we have a {power*100:.1f}% chance of detecting it with our sample size.")

if power >= 0.8:
    print(f"\n✓ Power is adequate (≥80% is standard)")
else:
    print(f"\n⚠️ Power is below the 80% standard - consider larger sample size")

print("="*70)

---

## Part 7: Final Report and Recommendations

### Task 7.1: Executive Summary

**✏️ YOUR TASK**: Write a brief executive summary (3-5 sentences) for the leadership team. This should be non-technical and focus on the business decision.

Consider including:
- What you tested
- What you found
- Your recommendation
- Expected business impact

**EXECUTIVE SUMMARY:**

_[Write your 3-5 sentence executive summary here]_

---

## Reflection Questions

**✏️ YOUR TASK**: Answer the following questions to deepen your understanding.

**1. What would happen if we used a two-tailed test instead of one-tailed? When would that be appropriate?**

_[Your answer]_

**2. What if we set α = 0.01 instead of 0.05? How would this change the decision? What's the tradeoff?**

_[Your answer]_

**3. If the test showed statistical significance but the confidence interval barely included the 2% target, what would you recommend? Why?**

_[Your answer]_

**4. What potential confounding variables or biases could affect this AB test in a real-world setting?**

_[Your answer]_

**5. How long should this test run in production? What factors would you consider?**

_[Your answer]_

---

## Submission Checklist

Before submitting, make sure you have:

- [ ] Recorded your random seed at the top
- [ ] Completed all code cells (filled in all `___` blanks)
- [ ] Stated your hypotheses clearly
- [ ] Calculated test statistic and p-value correctly
- [ ] Calculated confidence intervals
- [ ] Created all required visualizations
- [ ] Written your executive summary
- [ ] Answered all reflection questions
- [ ] Made a clear recommendation
- [ ] Explained both statistical and practical significance

---

## Congratulations! 🎉

You've completed a full AB test from hypothesis formulation through business recommendation. These skills are directly applicable to:

- Product development and optimization
- Marketing campaign evaluation
- User experience research
- Business strategy decisions
- Scientific research

**Key Takeaways:**
1. Always state hypotheses before collecting data
2. Statistical significance ≠ practical significance
3. Confidence intervals provide more information than p-values alone
4. Consider statistical power and sample size requirements
5. Business context matters as much as statistical results

Keep practicing, and you'll become an expert at data-driven decision making! 📊🚀